<a href="https://colab.research.google.com/github/ZainAli24/Quater_3/blob/main/Q3class3_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

   # **Retrieval-Augmented Generation (RAG)** : ka matlab yeh hai ke ek model pehle relevant information retrieve (ya dhoondh) karta hai aur phir us information ko use karke jawab generate karta hai.

In [1]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.9 MB/s eta 0:00:00


In [2]:
!pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.7 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model= 'gemini-1.5-flash'
)

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings.embed_query("What's our Q1 revenue?")

[0.01612485572695732,
 -0.03933568298816681,
 -0.002993757603690028,
 -0.033227454870939255,
 0.05022253096103668,
 0.03145093098282814,
 -0.00027826917357742786,
 -0.064089834690094,
 0.05403633043169975,
 0.02948707528412342,
 0.03588718920946121,
 0.024271711707115173,
 -0.035950273275375366,
 0.006472693756222725,
 0.029704688116908073,
 -0.05441908538341522,
 -0.03014720045030117,
 -0.013652290217578411,
 0.008419765159487724,
 -0.00975697860121727,
 -0.016383253037929535,
 0.061616990715265274,
 -0.028840864077210426,
 0.0015244809910655022,
 0.02118818275630474,
 0.02309279516339302,
 -0.009022949263453484,
 -0.022830545902252197,
 -0.03407009690999985,
 0.058693066239356995,
 -0.10280811786651611,
 0.06019517406821251,
 -0.03679637238383293,
 0.036128751933574677,
 0.0333784781396389,
 -0.05355016142129898,
 -0.02582419477403164,
 0.0417415127158165,
 -0.02746596373617649,
 0.025292860344052315,
 -0.02460378408432007,
 0.006439381279051304,
 -0.03763241693377495,
 -0.0278397221

In [6]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = TextLoader("Zain_document.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)


[Document(metadata={'source': 'Zain_document.txt'}, page_content="My name is Zain. I have study at Punjab College in Gujar Khan, and I used to go to college on a bike. Now, I am studying at PIAIC. In the evening, I also play cricket. My brother's name is Sameer. He flying kites in every evening for fun.\n\nIn the next month Sameer exams will start, Now days he fully focused for preparing exams. and he used mobile phone daily night in 9 a clock.\n\nWe live in Village Dulmi tumma, this village is very beautiful.")]


In [7]:
!pip install tqdm

In [12]:
from tqdm import tqdm
vectors = []

for doc in tqdm(docs):
  vector = embeddings.embed_query(doc.page_content)
  doc_id = str(hash(doc.page_content))
  # index.upsert(vectors=[(doc_id, vector)])
  index.upsert(vectors=[(doc_id, vector, {"text": doc.page_content})])
  vectors.append((doc_id, vector))



100%|██████████| 1/1 [00:00<00:00,  1.29it/s]


 ## **-) hash ka kaam yehi hai**: Kisi bhi value (string, number, etc.) ke liye ek unique ID (hash number) banata hai. Yeh ID har baar same input ke liye same hoti hai.
 - ###  Same value, same hash: Agar value wahi rahe, toh hash number bhi wahi rahega.
 - ### Different value, different hash: Agar value badle, toh hash number bhi badlega.

 - ### Hash ka kaam hota hai fast searching aur unique identification ke liye ek fixed-size number banana.

---
 - ##  **index**: Pinecone ka ek database object hai, jisme aap data store karte ho.
 - ## **upsert**: Yeh ek method hai jo:
>- Data ko store (insert) karta hai agar wo naya ho.
>- Data ko update karta hai agar wo pehle se maujood ho.**

In [10]:
from google.colab import userdata
api_keyees = userdata.get('pinecone_database')


In [11]:
from langchain_pinecone import PineconeVectorStore

from pinecone import Pinecone, ServerlessSpec

index_name = "quickrag7"

pc = Pinecone(api_key=api_keyees)

pc.create_index(
name=index_name,
 dimension=768, # Replace with your model dimensions
 metric="cosine", #  & Replace with your model metric
spec=ServerlessSpec(
cloud="aws",
region="us-east-1"
)
)

index = pc.Index(index_name)

vector_store= PineconeVectorStore(embedding=embeddings, index=index)

## **Simple Example of index = pc.Index(index_name)** :
- ## Socho, Pinecone ek building hai jisme bohot saare lockers hain.
- ## Har locker ka ek unique naam hai (e.g., "quickrag2").
- ## pc.Index("quickrag2") ka matlab hai:
>- ## Pinecone building ke andar jaakar "quickrag2" locker access karna.
- ## Jo locker access hota hai, uska ek control key (index) ban jata hai.
- ## Ab aap is control key (index) ko use karke locker ke andar cheezein daal aur nikaal ("vectors store, retrieve, aur update kar") sakte ho.


In [13]:
from langchain.vectorstores import Pinecone
retriever = Pinecone(index=index, embedding=embeddings, text_key="text")
retriever = vector_store.as_retriever()

<ipython-input-13-7332de4ed16a>:2: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  retriever = Pinecone(index=index, embedding=embeddings, text_key="text")


  ### ⚛ **retriever = Pinecone(index=index , embedding=embeddings, text_key="text") :**

>- ## Pinecone ke index (index) aur embeddings ko use karke ek retriever banata hai.
>- ## Retriever ka kaam hai text ko index se dhundhna aur relevant vectors retrieve karna.

>- ## Parameters:
>>- ### **index**: Aapka Pinecone locker (jo pehle access kiya tha).
>>- ### **embedding**: Embedding model (jo text ko vectors mein convert karta hai).
>>- ### **text_key** wo key hai jo Pinecone ko batati hai ke document ke andar asal text kahaan rakha hai


  ### ⚛ **retriever = vector_store.as_retriever():**

- ### Aapke paas ek tool (vector_store) hai jo Pinecone index ke saath kaam karta hai.
- ### Aap is tool ko ek retriever bana rahe hain, taki ye Pinecone ka data nikalne ke kaam aa sake.


In [14]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "refine",
    retriever = retriever
)


# **Kaise Kaam Hoga Hai Chain ke ander:**
> # 1. **Query Input:**
- Jab aap koi query dete hain (e.g., "What is LangChain?"), yeh query model ko forward hoti hai.

> # 2. **Retriever ka Kaam:**
- Retriever Pinecone database mein query ka similar vector search karta hai.
- Yeh search ke baad related documents ka data return karta hai.

> # 3. **LLM ka Kaam:**
- Related documents ko LLM ko pass kiya jata hai.
- LLM un documents ko read karta hai aur query ka jawab deta hai.

----

### **chain_type="refine" ka matlab hai:**

- Refine ka matlab hai pehle multiple steps mein jawab dhoondhna.
- Phir aakhri mein us jawab ko behtar aur accurate banana.
- Iska fayda yeh hai ke aapko sahi aur precise jawab milega.

- Dusre options bhi ho sakte hain, jaise "**map_reduce**", jo zyada data ko handle karta hai.

In [18]:
query = "Who is Zain and exam"
response = qa_chain.run(query)  # First item of the tuple
print(response)

Zain is a person who studied at Punjab College in Gujar Khan and now studies at PIAIC.  He rides a bike to college.  The exam mentioned is Sameer's exam, which is starting next month.
